In [ ]:
pip install lime

In [ ]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
stopwords_list ='stopwords-bn.txt'

**Data Preparation**

In [ ]:
# data = pd.read_excel('corpus_all.xlsx')
train_df = pd.read_excel('/content/train_emotion.xlsx')
val_df = pd.read_excel('/content/test_emotion.xlsx')

In [ ]:
# import pandas as pd
# from sklearn.utils import resample

# # Assuming your target variable is named 'target'
# classes = data['comment_type'].unique()
# # Create an empty list to store the sampled data
# sampled_data = []

# # Loop through each class and take 600 random samples
# for class_value in classes:
#     class_data = data[data['comment_type'] == class_value]
#     sampled_class_data = resample(class_data, replace=False, n_samples=384, random_state=42)
#     sampled_data.append(sampled_class_data)

# # Concatenate the sampled data into a single DataFrame
# sampled_data = pd.concat(sampled_data, ignore_index=True)

# # Save the sampled data to an Excel file
# sampled_data.to_excel('sampled_data.xlsx', index=False)

In [ ]:
# data = pd.read_excel('sampled_data.xlsx')

In [ ]:
# data.describe()

In [ ]:
class_values_train = train_df['comment_type'].unique()
print("Unique class values train:", class_values_train)

class_values_test = val_df['comment_type'].unique()
print("Unique class values test:", class_values_test)
# Count the number of instances for each class
class_counts_train = train_df['comment_type'].value_counts()
print("\nClass counts train:")
print(class_counts_train)
# Count the number of instances for each class
class_counts_test = val_df['comment_type'].value_counts()
print("\nClass counts test:")
print(class_counts_test)

In [ ]:
from utils import cleaned_reviews,stopwords_info,stopword_removal,process_reviews

In [ ]:
#data['cleaned'] = data['comment'].apply(process_reviews,stopwords = stopwords_list,removing_stopwords = True)
train_df['cleaned'] = train_df['comment'].apply(process_reviews,stopwords = stopwords_list,removing_stopwords = False)

In [ ]:
train_df[['cleaned','comment_type']].to_excel('clean_reviews_train.xlsx')

In [ ]:
train_df.head()

,comment,comment_type,cleaned
0,এরপর যেই তারা দাফনের কাজ শুরু করবে ঠিক তখনি কব...,fear,এরপর যেই তারা দাফনের কাজ শুরু করবে ঠিক তখনি কব...
1,আমি মন্তব্য করতে দ্বিধা বোধ করছি কারণ আমি পাইথ...,fear,আমি মন্তব্য করতে দ্বিধা বোধ করছি কারণ আমি পাইথ...
2,আপনারা এসব করে হীনোমোনোত্তর প্রমান দিচ্ছন ....,disgust,আপনারা এসব করে হীনোমোনোত্তর প্রমান দিচ্ছন ...
3,ঈশ্বর আমি এখানে আঠা আকাশ থেকে সত্যিই উঁচু,surprise,ঈশ্বর আমি এখানে আঠা আকাশ থেকে সত্যিই উঁচু
4,আমি প্রাক্তন থেকে দূরে থাকাকে আনন্দিত বোধ করি,joy,আমি প্রাক্তন থেকে দূরে থাকাকে আনন্দিত বোধ করি


In [ ]:
train_df = train_df.drop(columns=["comment"])

In [ ]:
train_df.head()

In [ ]:
val_df['cleaned'] = val_df['comment'].apply(process_reviews,stopwords = stopwords_list,removing_stopwords = False)

In [ ]:
val_df[['cleaned','comment_type']].to_excel('clean_reviews_test.xlsx')

In [ ]:
val_df = val_df.drop(columns=["comment"])

In [ ]:
## split to train and val
# train_df, val_df = train_test_split(data, test_size=0.2, random_state=4021)
# val_df.reset_index(drop=True)

**Model Train**

In [ ]:
# tfidf_vc = TfidfVectorizer(min_df = 10, max_features = 100000, analyzer = "word", ngram_range = (1, 2), lowercase = True)
tfidf_vc = TfidfVectorizer(min_df = 10, max_features = 100000, analyzer = "word", ngram_range = (1, 2), lowercase = True)
train_vc = tfidf_vc.fit_transform(train_df["cleaned"])
val_vc = tfidf_vc.transform(val_df["cleaned"])


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
param_grid = {
    'C': [0.1, 0.5, 1, 2],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

In [ ]:
model = LogisticRegression()

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_vc, train_df.comment_type)

In [ ]:
lr_best_model = grid_search.best_estimator_
lr_best_params = grid_search.best_params_

In [ ]:
import pickle

with open('lr_best_model.pkl', 'wb') as file:
    pickle.dump(lr_best_model, file)

In [ ]:
val_pred = lr_best_model.predict(val_vc)

In [ ]:
val_acc = accuracy_score(val_df.comment_type, val_pred)
val_precision = precision_score(val_df.comment_type, val_pred, average="macro")
val_recall = recall_score(val_df.comment_type, val_pred, average="macro")
val_f1 = f1_score(val_df.comment_type, val_pred, average="macro")

In [ ]:
print('LR')
print("Accuracy:", val_acc)
print("Precision:", val_precision)
print("Recall:", val_recall)
print("F1 score:", val_f1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
model = RandomForestClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_vc, train_df.comment_type)

In [ ]:
rf_best_model = grid_search.best_estimator_

In [ ]:
print(grid_search.best_params_)

In [ ]:
with open('rf_best_model.pkl', 'wb') as file:
    pickle.dump(rf_best_model, file)

In [ ]:
with open('rf_best_model.pkl', 'rb') as file:
    best_model_rf = pickle.load(file)

val_pred_rf = rf_best_model.predict(val_vc)

val_acc_rf = accuracy_score(val_df.comment_type, val_pred_rf)
val_precision_rf = precision_score(val_df.comment_type, val_pred_rf, average="macro")
val_recall_rf = recall_score(val_df.comment_type, val_pred_rf, average="macro")
val_f1_rf = f1_score(val_df.comment_type, val_pred_rf, average="macro")

print("Random Forest:")
print("- Accuracy:", val_acc_rf)
print("- Precision:", val_precision_rf)
print("- Recall:", val_recall_rf)
print("- F1 score:", val_f1_rf)

In [ ]:
from sklearn.svm import SVC

param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 1],
    'gamma': ['scale', 'auto']
}

In [ ]:
model = SVC()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_vc, train_df.comment_type)

In [ ]:
best_model_svm = grid_search.best_estimator_

with open('best_model_svm.pkl', 'wb') as file:
    pickle.dump(best_model_svm, file)

In [ ]:
with open('best_model_svm.pkl', 'rb') as file:
    best_model_svm = pickle.load(file)

val_pred_svm = best_model_svm.predict(val_vc)

val_acc_svm = accuracy_score(val_df.comment_type, val_pred_svm)
val_precision_svm = precision_score(val_df.comment_type, val_pred_svm, average="macro")
val_recall_svm = recall_score(val_df.comment_type, val_pred_svm, average="macro")
val_f1_svm = f1_score(val_df.comment_type, val_pred_svm, average="macro")

print("\nSVM:")
print("- Accuracy:", val_acc_svm)
print("- Precision:", val_precision_svm)
print("- Recall:", val_recall_svm)
print("- F1 score:", val_f1_svm)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

In [ ]:
model = KNeighborsClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_vc, train_df.comment_type)

In [ ]:
best_model_knn = grid_search.best_estimator_

In [ ]:
with open('best_model_knn.pkl', 'wb') as file:
    pickle.dump(best_model_knn, file)

In [ ]:
with open('best_model_knn.pkl', 'rb') as file:
    best_model_knn = pickle.load(file)

val_pred_knn = best_model_knn.predict(val_vc)

val_acc_knn = accuracy_score(val_df.comment_type, val_pred_knn)
val_precision_knn = precision_score(val_df.comment_type, val_pred_knn, average="macro")
val_recall_knn = recall_score(val_df.comment_type, val_pred_knn, average="macro")
val_f1_knn = f1_score(val_df.comment_type, val_pred_knn, average="macro")

print("\nKNN:")
print("- Accuracy:", val_acc_knn)
print("- Precision:", val_precision_knn)
print("- Recall:", val_recall_knn)
print("- F1 score:", val_f1_knn)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'alpha': [0.5, 1.0, 2.0],
    'fit_prior': [True, False]
}

In [ ]:
model = MultinomialNB()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_vc, train_df.comment_type)


In [ ]:
print(grid_search.best_params_)

In [ ]:
best_model_mnb = grid_search.best_estimator_

with open('best_model_mnb.pkl', 'wb') as file:
    pickle.dump(best_model_mnb, file)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model_mnb = MultinomialNB()
model_mnb.fit(train_vc, train_df.comment_type)
val_pred_mnb = model_mnb.predict(val_vc)

In [ ]:
with open('best_model_mnb.pkl', 'rb') as file:
    best_model_mnb = pickle.load(file)

val_pred_mnb = best_model_mnb.predict(val_vc)

val_acc_mnb = accuracy_score(val_df.comment_type, val_pred_mnb)
val_precision_mnb = precision_score(val_df.comment_type, val_pred_mnb, average="macro")
val_recall_mnb = recall_score(val_df.comment_type, val_pred_mnb, average="macro")
val_f1_mnb = f1_score(val_df.comment_type, val_pred_mnb, average="macro")

print("\nMultinomial Naive Bayes:")
print("- Accuracy:", val_acc_mnb)
print("- Precision:", val_precision_mnb)
print("- Recall:", val_recall_mnb)
print("- F1 score:", val_f1_mnb)

In [ ]:
# Get the indices of the correct predictions made by the Multinomial Naive Bayes model
correct_indices = [i for i in range(len(val_pred_mnb)) if val_pred_mnb[i] == val_df["comment_type"].iloc[i]]

# Print the corresponding rows from val_df
val_df.iloc[correct_indices]

In [ ]:
correct_predictions = val_df[val_df["comment_type"] == val_pred_mnb]
correct_predictions.to_csv("correct_predictions.csv", index=False)

In [ ]:
corr_mnb = pd.read_csv('correct_predictions.csv')
corr_mnb

In [ ]:
val_df.to_csv("val_df.csv", index=False)

**LIME**

In [ ]:
class_names = (train_df['comment_type'].unique())

In [ ]:
class_names

In [ ]:
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline

In [ ]:
index_of_ = val_df.index.get_loc(1504)
print(index_of_)

In [ ]:
for index in val_df.index:
    print(index)

In [ ]:
print(val_df.index[1343])

In [ ]:
index_of_ = val_df.index.get_loc(4289)
idx = val_df.index[index_of_]
c = make_pipeline(tfidf_vc, model_mnb)
class_names = class_names
explainer = LimeTextExplainer(class_names = class_names)
exp = explainer.explain_instance(val_df["cleaned"][idx], c.predict_proba, num_features = 5)

print("Comment: \n", val_df["cleaned"][idx])
print("Probability (happy) =", c.predict_proba([val_df["cleaned"][idx]])[0, 0])
print("Probability (sad) =", c.predict_proba([val_df["cleaned"][idx]])[0, 1])
print("Probability (angry) =", c.predict_proba([val_df["cleaned"][idx]])[0, 2])
print("Probability (surprise) =", c.predict_proba([val_df["cleaned"][idx]])[0, 3])
print("Probability (disgust) =", c.predict_proba([val_df["cleaned"][idx]])[0, 4])
#print("Probability (fear) =", c.predict_proba([val_df["cleaned"][idx]])[0, 5])
#print("True Class is:", class_names[val_df["comment_type"][idx]])

In [ ]:
exp = explainer.explain_instance(val_df["cleaned"][idx], c.predict_proba, num_features=6, top_labels=2)
print(exp.available_labels())

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp.show_in_notebook(text=val_df["cleaned"][idx], labels=(3,))

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp.show_in_notebook(text=val_df["cleaned"][idx], labels=(1,))

In [ ]:
idx = val_df.index[4]
c = make_pipeline(tfidf_vc, model_mnb)
class_names = class_names
explainer = LimeTextExplainer(class_names = class_names)
exp = explainer.explain_instance(val_df["cleaned"][idx], c.predict_proba, num_features = 10)

print("Comment: \n", val_df["cleaned"][idx])
print("Probability (fear) =", c.predict_proba([val_df["cleaned"][idx]])[0, 0])
print("Probability (sad) =", c.predict_proba([val_df["cleaned"][idx]])[0, 1])
print("Probability (happy) =", c.predict_proba([val_df["cleaned"][idx]])[0, 2])
print("Probability (angry) =", c.predict_proba([val_df["cleaned"][idx]])[0, 3])
print("Probability (surprise) =", c.predict_proba([val_df["cleaned"][idx]])[0, 4])
print("Probability (disgust) =", c.predict_proba([val_df["cleaned"][idx]])[0, 5])
#print("True Class is:", class_names[val_df["comment_type"][idx]])